## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from pprint import pprint
import sys
import matplotlib.pyplot as plt
# Import API key
sys.path.append('..')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hasaki,JP,35.73,140.83,66.20,88,75,2.24,light intensity shower rain
1,1,Karratha,AU,-20.74,116.85,56.84,37,93,15.86,overcast clouds
2,2,Yellowknife,CA,62.46,-114.35,50.00,76,75,18.34,broken clouds
3,3,Mataura,NZ,-46.19,168.86,48.99,96,100,11.01,light rain
4,4,Sulangan,PH,11.14,123.72,82.02,78,99,8.23,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like? '))
max_temp = float(input('What is the maximum temperature you would like? '))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preffered_city = city_data_df.loc[(city_data_df['Max Temp']>=min_temp) & (city_data_df['Max Temp']<=max_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preffered_city.isnull().sum()

Unnamed: 0             0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preffered_city.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location":""
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_name = hotel['results'][0]['name']
        hotel_df.loc[index,'Hotel Name'] = hotel_name
    except:
        print('Hotel name not found for city: ',row['City'])
        pass

        

Hotel name not found for city:  Avera
Hotel name not found for city:  Carnot
Hotel name not found for city:  Mangai
Hotel name not found for city:  Camopi
Hotel name not found for city:  Kalakamati
Hotel name not found for city:  Gamba
Hotel name not found for city:  Poya
Hotel name not found for city:  Boende
Hotel name not found for city:  Zemio


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Grand-Lahou,CI,75.61,clear sky,5.14,-5.02,Hotel Beatitude de Grand Lahou
9,Saint-Philippe,RE,71.60,overcast clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
11,Atuona,PF,78.40,few clouds,-9.80,-139.03,Villa Enata
18,Georgetown,MY,78.80,light rain,5.41,100.34,Cititel Penang
28,Payo,ID,72.86,overcast clouds,-3.75,103.64,Mess keban agung


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dt>{Hotel Name}</dt>
<dt><b>City</b></dt><dt>{City}</dt>
<dt><b>Country Code</b></dt><dt>{Country}</dt>
<dt><b>Weather Description</b></dt><dt>{Current Description}</dt>
<dt><b>Max Temp</b></dt><dt>{Max Temp} °F</dt>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,weights=max_temp, max_intensity=300, dissipating =False, point_radius=4 )
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height=&#39;420px&#39;))